In [ ]:
import os
os.remove('snorkel.db')

In [1]:
from snorkel import SnorkelSession
session = SnorkelSession()

import os, sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

In [2]:
import os
from snorkel.parser import CorpusParser
from snorkel.parser import HTMLParser
from snorkel.parser import OmniParser

doc_parser = HTMLParser(path=os.environ['SNORKELHOME'] + '/tutorials/tables/sandbox/diseases.xhtml')
context_parser = OmniParser()
cp = CorpusParser(doc_parser, context_parser, max_docs=1)

In [3]:
%time corpus = cp.parse_corpus(name='Sandbox', session=session)
print "%d Phrases in %s" % (
    len([phrase for doc in corpus.documents for phrase in doc.phrases]), corpus)

[========================================] 100%

CPU times: user 330 ms, sys: 22.4 ms, total: 352 ms
Wall time: 1.07 s
31 Phrases in Corpus (Sandbox)


In [4]:
# for cell in corpus.documents[0].cells: print cell.text[:10]

In [5]:
# for phrase in [phrase for doc in corpus.documents for phrase in doc.phrases]:
#     print phrase

In [6]:
# (a,b,c) = (2,(3 if True else 5) ,4)

In [7]:
from snorkel.models import candidate_subclass

# Year = candidate_subclass('Year', ['year'])
# Temp = candidate_subclass('Temp', ['temp'])
# Disease = candidate_subclass('Disease', ['disease'])
# Part = candidate_subclass('Part', ['part'])
Disease_Year = candidate_subclass('Disease_Year', ['disease','year'])

In [8]:
from snorkel.candidates import OmniNgrams
from hardware_utils import OmniNgramsPart, OmniNgramsTemp

omni_ngrams = OmniNgrams(n_max=3, split_tokens=None)
# omni_part = OmniNgramsPart(n_max=3, split_tokens=None)
# omni_temp = OmniNgramsTemp(n_max=3, split_tokens=None)

In [9]:
diseases = ['viruses', 'coughs', 'colds', 'brain', 'cancer', 'shingles', 'warts',
              'brain cancer', 'disease', 'location', 'polio', 'chicken pox', 
              'bubonic plague', 'plague', 'scurvy', 'infectious diseases', 
              'arthritis', 'yellow fever', 'fever', 'hypochondria', 'pneumonia',
              'whooping cough']
print "Loaded %d diseases." % len(diseases)

Loaded 22 diseases.


In [10]:
from snorkel.matchers import DictionaryMatch, RegexMatchEach, RegexMatchSpan, DateMatcher

disease_matcher = DictionaryMatch(d=diseases, ignore_case=True)
year_matcher = DateMatcher()
# part_matcher = RegexMatchEach(rgx='BC.*')
# temp_matcher = RegexMatchSpan(rgx=r'-[5-7][05]')

In [11]:
from snorkel.candidates import CandidateExtractor

# ce = CandidateExtractor(Part, [omni_part], [part_matcher])
# ce = CandidateExtractor(Disease, [omni_ngrams], [disease_matcher])
# ce = CandidateExtractor(Temp, [omni_temp], [temp_matcher])
ce = CandidateExtractor(Disease_Year, 
                        [omni_ngrams, omni_ngrams], 
                        [disease_matcher, year_matcher])

In [12]:
%time candidates = ce.extract(corpus.documents, 'Sandbox Candidates', session)
print "%s contains %d Candidates" % (candidates, len(candidates))

[========================================] 100%

CPU times: user 102 ms, sys: 5.43 ms, total: 107 ms
Wall time: 111 ms
Candidate Set (Sandbox Candidates) contains 36 Candidates


In [13]:
for c in candidates: print c.disease

Span("Hypochondria", parent=70, chars=[0,11], words=[0,0])
Span("Hypochondria", parent=70, chars=[0,11], words=[0,0])
Span("coughs", parent=2, chars=[18,23], words=[4,4])
Span("coughs", parent=2, chars=[18,23], words=[4,4])
Span("warts", parent=24, chars=[34,38], words=[7,7])
Span("warts", parent=24, chars=[34,38], words=[7,7])
Span("Brain", parent=3, chars=[0,4], words=[0,0])
Span("Brain", parent=3, chars=[0,4], words=[0,0])
Span("Scurvy", parent=58, chars=[0,5], words=[0,0])
Span("Scurvy", parent=58, chars=[0,5], words=[0,0])
Span("Infectious diseases", parent=15, chars=[9,27], words=[3,4])
Span("Infectious diseases", parent=15, chars=[9,27], words=[3,4])
Span("Arthritis", parent=64, chars=[0,8], words=[0,0])
Span("Arthritis", parent=64, chars=[0,8], words=[0,0])
Span("Brain Cancer", parent=3, chars=[0,11], words=[0,1])
Span("Brain Cancer", parent=3, chars=[0,11], words=[0,1])
Span("viruses", parent=2, chars=[9,15], words=[2,2])
Span("viruses", parent=2, chars=[9,15], words=[2,2])
Sp

In [16]:
c = candidates[19]
print c

Disease_Year(Span("whooping cough", parent=57, chars=[0,13], words=[0,1]), Span("1901", parent=60, chars=[0,3], words=[0,0]))


In [22]:
from snorkel.lf_helpers import get_row_ngrams, get_right_ngrams

# print [cell.text[:8] for cell in _get_aligned_cells(root_phrase.cell, axis, infer=infer)]
# print [phrase.text for cell in _get_aligned_cells(root_phrase.cell, axis, infer=infer) for phrase in cell.phrases]
print list(get_row_ngrams(c.disease, n_max=1, direct=True, infer=True))
print list(get_right_ngrams(c.disease, n_max=1))

[u'i', u'do', u"n't", u'like', u'chicken', u'pox', u'or', u'pizza', u'.', u'shingles', u'is', u'also', u'bad', u'.', u'%', u'2001']
[]


In [ ]:
# for c in candidates: print c

In [ ]:
# for c in candidates:
#     print c
c = candidates[14]
print c
print c.disease.parent.document
print c.disease.parent.table
# print c.disease.parent.row
# print c.disease.parent.col
print c.disease.parent.cell
print c.disease.parent

In [ ]:
from snorkel.models import CandidateSet
train = CandidateSet(name='train', candidates=candidates[:25])
dev = CandidateSet(name='test', candidates=candidates[25:])
print len(train)
print train[0]
print len(dev)
print dev[0]

In [ ]:
print len(train)

In [ ]:
from snorkel.annotations import FeatureManager

feature_manager = FeatureManager()
%time F_train = feature_manager.create(session, train, 'Sandbox Features')
F_train

# from snorkel.features import get_span_feats
# c = candidates[10]
# %prun for feat in get_span_feats(c): print feat

In [ ]:
# other_manager = FeatureManager()
# %time G_train = feature_manager.create(session, dev, 'Other Features')
# G_train

In [ ]:
# F_train + G_train

In [ ]:
c = candidates[25]
print c
print c.disease.parent.row_num
print c.disease.parent.row
print c.disease.parent.col_num
print c.disease.parent.col

In [ ]:
from snorkel.utils import get_keys_by_candidate

print candidates[22]
print "Limited number of features being shown:"
for f in get_keys_by_candidate(F_train, F_train.get_candidate(22))[:]: print f

In [ ]:
# c = candidates[0]
# print c
# print c.year
# print c.year.parent
# print c.year.char_start
# print c.year.char_end
# print c.year.parent.words
# print c.year.parent.char_offsets
# print c.year.parent.lemmas
# print c.year.parent.pos_tags
# print c.year.parent.ner_tags
# print c.year.parent.dep_parents
# print c.year.parent.dep_labels
# print c.year.get_word_start()
# print c.year.get_word_end()
# # print get_row_ngrams(c.year)
# # print get_row_ngrams(c.year, infer=True)
# %time for i in range(1000): _get_aligned_cells((c.year).parent.cell, 'col', infer=True)

In [ ]:
# from snorkel.models import Span, ImplicitSpan, TemporarySpan

# print isinstance(c.year, TemporarySpan)

In [ ]:
# from snorkel.lf_helpers import get_between_ngrams, get_left_ngrams, get_right_ngrams
# from snorkel.lf_helpers import contains_token, contains_regex
# from snorkel.lf_helpers import get_phrase_ngrams, get_cell_ngrams, get_neighbor_cell_ngrams
# from snorkel.lf_helpers import get_row_ngrams, get_col_ngrams, get_aligned_ngrams
# from snorkel.lf_helpers import same_document, same_table, same_cell, same_phrase
# from snorkel.lf_helpers import _get_aligned_cells, _get_nonempty_cell

In [ ]:
# print get_left_ngrams(c.disease)
# print get_right_ngrams(c.disease)
# print contains_token(c, 'plague')
# print contains_regex(c, r'pla')
# print same_document(c)
# print same_table(c)
# print same_cell(c)
# print same_phrase(c)
# print get_phrase_ngrams(c.disease, n_min=1, n_max=3, case_sensitive=True)
# print get_cell_ngrams(c.disease, attrib='pos_tags')
# print get_neighbor_cell_ngrams(c.disease, dist=2, directions=True)
# print get_row_ngrams(c.disease)
# print get_col_ngrams(c.disease)
# print get_aligned_ngrams(c.disease)
# print get_aligned_ngrams(c.disease, infer=True)

In [ ]:
# from hardware_utils import expand_implicit_text 

# for part in expand_implicit_text(''.join(['BC856/857/858', '/', '859/860'])): print part

In [ ]:
# def foo(n):
#     phrase = 'repeat me'
#     pmul = phrase * n
#     pjoi = ''.join([phrase for x in xrange(n)])
#     pinc = ''
#     for x in xrange(n):
#         pinc += phrase
#     del pmul, pjoi, pinc

In [ ]:
# %lprun -f foo foo(1000)

In [ ]:
# from hardware_utils import get_gold_dict

# filename = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
# gold_dict = get_gold_dict(filename, 'stg_temp_min')

In [ ]:
# print len(gold_dict.values())
# print gold_dict.values().count(1)

In [ ]:
# from hardware_utils import count_hardware_labels

# filename = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
# %time (certain, maybe) = count_hardware_labels(loader, candidates, filename, attrib='stg_temp_min', attrib_class='temp')

In [ ]:
%time 
for i in range(10000): 
    for j in range(100):
        1 + 1